## <center>Find your Plan</center>

### <font color='maroon'>Problem Statement</font>

**Help custometrs with right plan and rigth device based on the requirement**.

#### <font color='maroon'>Importing Libraries</font> 

In [1]:
#Install OpenAI library
!pip install -U -q openai tenacity

In [2]:
import os, json, ast
import openai
from tenacity import retry, wait_random_exponential, stop_after_attempt
import pandas as pd
from IPython.display import display, HTML

#### <font color='maroon'>Reading the OpenAI API key</font> 

In [3]:
# Read the OpenAI API key
openai.api_key = os.getenv("OPENAI_API_KEY")

#### <font color='maroon'>Reading the CSV</font> 

In [4]:
# Set the display width to control the output width
pd.set_option('display.width', 100)
# Read the dataset and read the Laptop Dataset
df = pd.read_csv('telecomData1.csv')
df

,Brand,Plan Name,Monthly Cost (USD),Data Allowance,Talk Time,SMS,Max Download speed(Mbps),Max Upload Speed(Mbps),Cloud Storage,Hotspot Data(GB),Plan Description
0,StraightTalk,Platinum unlimited,65,Unlimited,Unlimited,Unlimited,195,31,Yes,Unlimited,Platinum plan under Straight talk brand with u...
1,StraightTalk,Gold Unlimited,55,Unlimited,Unlimited,Unlimited,195,31,Yes,30,Gold plan under Straight talk brand with unlim...
2,StraightTalk,Silver Unlimited,45,Unlimited,Unlimited,Unlimited,149,25,No,10,Silver plan under Straight talk brand with unl...
3,StraightTalk,Bronze plan,35,Unlimited,Unlimited,Unlimited,149,25,No,0,Silver plan under Straight talk brand with unl...
4,TotalWireless,Total5G+ Unlimited,65,Unlimited,Unlimited,Unlimited,195,31,No,Unlimited,Total 5g+ unlmited plan under Total Wireless B...
5,TotalWireless,Gold Unlimited,55,Unlimited,Unlimited,Unlimited,195,31,No,15,Total 5g unlmited plan under Total Wireless Br...
6,TotalWireless,Silver Unlimited,40,Unlimited,Unlimited,Unlimited,149,25,No,5,Total 5g+ unlmited plan under Total Wireless B...


In [13]:
# Set the display width to control the output width
pd.set_option('display.width', 100)
# Read the dataset and read the Laptop Dataset
df2 = pd.read_csv('laptop_data.csv')
df2

,Brand,Model Name,Core,CPU Manufacturer,Clock Speed,RAM Size,Storage Type,Display Type,Display Size,Graphics Processor,Screen Resolution,OS,Laptop Weight,Special Features,Warranty,Average Battery Life,Price,Description
0,Dell,Inspiron,i5,Intel,2.4 GHz,8GB,SSD,LCD,"15.6""",Intel UHD,1920x1080,Windows 10,2.5 kg,Backlit Keyboard,1 year,6 hours,"35,000",The Dell Inspiron is a versatile laptop that c...
1,MSI,GL65,i7,Intel,2.6 GHz,16GB,HDD+SSD,IPS,"15.6""",NVIDIA GTX,1920x1080,Windows 10,2.3 kg,RGB Keyboard,2 years,4 hours,"55,000",The MSI GL65 is a high-performance laptop desi...
2,HP,EliteBook,i7,Intel,2.8 GHz,16GB,SSD,LED,"14""",Intel UHD,1920x1080,Windows 11,1.5 kg,Fingerprint Sensor,3 years,8 hours,"90,000",The HP EliteBook is a premium laptop designed ...
3,Lenovo,IdeaPad,i3,Intel,2.1 GHz,8GB,HDD,TN,"15.6""",Intel UHD,1366x768,Windows 10,2.2 kg,Dolby Audio,1 year,5 hours,"25,000",The Lenovo IdeaPad is a versatile laptop that ...
4,ASUS,ZenBook Pro,i9,Intel,3.1 GHz,64GB,SSD,OLED,"15.6""",NVIDIA RTX,3840x2160,Windows 10,1.8 kg,NanoEdge Display,2 years,7 hours,"200,000",The ASUS ZenBook Pro is a high-end laptop that...
5,Acer,Predator,i7,Intel,2.8 GHz,16GB,SSD,IPS,"17.3""",NVIDIA GTX,1920x1080,Windows 10,3.2 kg,Dual Cooling Fans,1 year,5 hours,"80,000",The Acer Predator is a powerhouse laptop desig...
6,Microsoft,Surface Laptop,i5,Intel,1.6 GHz,8GB,SSD,PixelSense,"13.5""",Intel Iris Plus,2256x1504,Windows 11,1.3 kg,Touchscreen Display,1 year,10 hours,"90,000",The Microsoft Surface Laptop is a premium devi...
7,Lenovo,ThinkPad,Ryzen 7,AMD,3.0 GHz,16GB,SSD,IPS,"14""",NVIDIA GTX,2560x1440,Linux,1.6 kg,Backlit Keyboard,3 years,6 hours,"60,000",The Lenovo ThinkPad is a powerful laptop desig...
8,HP,Pavilion,i5,Intel,2.3 GHz,12GB,HDD,LCD,"15.6""",Intel UHD,1366x768,Windows 10,2.1 kg,B&O Audio,1 year,4 hours,"30,000",The HP Pavilion is a budget-friendly laptop th...
9,ASUS,ROG Strix G,i7,Intel,2.9 GHz,16GB,SSD,IPS,"17.3""",NVIDIA RTX,1920x1080,Windows 10,2.9 kg,Aura Sync RGB Keyboard,2 years,5 hours,"85,000",The ASUS ROG Strix G is a high-performance gam...


#### <font color='maroon'>Approach:</font> 

1. **Conversation and Information Gathering**: The chatbot will utilize language models to understand and generate natural responses. Through a conversational flow, it will ask relevant questions to gather information about the user's requirements.
2. **Information Extraction**: Once the essential information is collected, rule-based functions come into play, extracting top 3 laptops that best matches the user's needs.
3. **Personalized Recommendation**: Leveraging this extracted information, the chatbot engages in further dialogue with the user, efficiently addressing their queries and aiding them in finding the perfect laptop solution.

#### <font color='maroon'>Original Code:</font> 

The original code manages a dialogue system where a chatbot interacts with users to help them choose laptops. The conversation flow involves checking user inputs for moderation, confirming intents, extracting variables, and providing recommendations based on the user's input.

#### <font color='maroon'>Changes Made:</font> 

##### <font color='maroon'>Function Calling API Integration:</font> 

The primary enhancement is the integration of the Function Calling API. This API helps determine if the user intends to exit the conversation based on the context and recent messages.

New Function: `get_chat_completions_with_exit_check`

- This function uses the Function Calling API to analyze the conversation and decide if the user wants to exit.
- If an exit intent is detected, it calls the exit_chat function, which gracefully ends the conversation.

##### <font color='maroon'>Refined Conversation Flow:</font> 

The conversation flow has been enhanced to make interactions more natural. The system now continuously checks for exit intents using the `get_chat_completions_with_exit_check` function. If the user wishes to end the conversation, the system exits gracefully.

In [19]:
MODEL = 'gpt-3.5-turbo'

In [6]:
def get_user_preferences():
    """Collects user preferences interactively."""
    print("Welcome! Let's find the best telecom plan for you.")
    budget = float(input("What is your budget (in USD)? "))
    data_required = input("How much data do you need? (e.g., 10GB, Unlimited): ")
    sms_required = input("How many SMS do you need? (e.g., 500, Unlimited): ")
    download_speed = input("Minimum required download speed (Mbps): ")
    upload_speed = input("Minimum required upload speed (Mbps): ")
    cloud_storage = input("Do you need cloud storage? (Yes/No): ")
    hotspot_data = input("How much hotspot data do you need? (e.g., 5GB, Unlimited): ")
    return budget, data_required, sms_required, download_speed, upload_speed, cloud_storage, hotspot_data

In [7]:
conversation = get_user_preferences()
print(conversation)

Welcome! Let's find the best telecom plan for you.


What is your budget (in USD)?  50
How much data do you need? (e.g., 10GB, Unlimited):  10
How many SMS do you need? (e.g., 500, Unlimited):  Unlimited
Minimum required download speed (Mbps):  100
Minimum required upload speed (Mbps):  25
Do you need cloud storage? (Yes/No):  Yes
How much hotspot data do you need? (e.g., 5GB, Unlimited):  10


(50.0, '10', 'Unlimited', '100', '25', 'Yes', '10')


In [9]:
def generate_prompt(budget, data_required, sms_required, download_speed, upload_speed, cloud_storage, hotspot_data, df):
    """Generates a prompt for OpenAI to suggest the best plans."""
    plan_data = df.to_dict(orient='records')
    prompt = f"""
    You are a telecom expert. A user is looking for a mobile plan based on their preferences:
    - Budget: ${budget} USD
    - Minimum Data Required: {data_required}
    - Minimum SMS Required: {sms_required}
    - Minimum Download Speed: {download_speed} Mbps
    - Minimum Upload Speed: {upload_speed} Mbps
    - Cloud Storage Required: {cloud_storage}
    - Minimum Hotspot Data: {hotspot_data}
    
    Given the following plan options:
    {plan_data}
    
    Suggest the top 3 most suitable plans based on the user's preferences.
    Provide the results in JSON format with fields: Brand, Plan Name, Monthly Cost (USD), Data Allowance, Talk Time, SMS, Max Download Speed (Mbps), Max Upload Speed (Mbps), Cloud Storage, Hotspot Data (GB), and Plan Description.
    """
    return prompt


In [23]:
# Define a Chat Completions API call
def get_openai_response(prompt):
    """Fetches response from OpenAI API using ChatCompletion."""
    client = openai.OpenAI()  # ✅ NEW - Use OpenAI Client

    response = client.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": "You are a telecom expert who suggests plans based on user needs."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.7
    )
    return response.choices[0].message.content


In [24]:
csv_path = "telecomData1.csv"

In [25]:
def suggest_plans(csv_path):
    """Main function to process user input interactively and get the best plans using OpenAI."""
    df = pd.read_csv(csv_path)
    budget, data_required, sms_required, download_speed, upload_speed, cloud_storage, hotspot_data = get_user_preferences()
    prompt = generate_prompt(budget, data_required, sms_required, download_speed, upload_speed, cloud_storage, hotspot_data, df)
    response = get_openai_response(prompt)
    print("\nHere are the top 3 recommended plans for you:")
    print(response)

In [26]:
suggest_plans(csv_path)

Welcome! Let's find the best telecom plan for you.


What is your budget (in USD)?  50
How much data do you need? (e.g., 10GB, Unlimited):  Unlimited
How many SMS do you need? (e.g., 500, Unlimited):  Unlimited
Minimum required download speed (Mbps):  100
Minimum required upload speed (Mbps):  25
Do you need cloud storage? (Yes/No):  Yes
How much hotspot data do you need? (e.g., 5GB, Unlimited):  10



Here are the top 3 recommended plans for you:
```json
[
    {
        "Brand": "StraightTalk",
        "Plan Name": "Silver Unlimited",
        "Monthly Cost (USD)": 45,
        "Data Allowance": "Unlimited",
        "Talk Time": "Unlimited",
        "SMS": "Unlimited",
        "Max Download Speed (Mbps)": 149,
        "Max Upload Speed (Mbps)": 25,
        "Cloud Storage": "No",
        "Hotspot Data (GB)": "10",
        "Plan Description": "Silver plan under Straight talk brand with unlimited data, talktime and SMS access along with 10 GB Hotspot access"
    },
    {
        "Brand": "TotalWireless",
        "Plan Name": "Silver Unlimited",
        "Monthly Cost (USD)": 40,
        "Data Allowance": "Unlimited",
        "Talk Time": "Unlimited",
        "SMS": "Unlimited",
        "Max Download Speed (Mbps)": 149,
        "Max Upload Speed (Mbps)": 25,
        "Cloud Storage": "No",
        "Hotspot Data (GB)": "5",
        "Plan Description": "Total 5g+ unlmited plan under Total Wi

In [27]:
suggest_plans(csv_path)

Welcome! Let's find the best telecom plan for you.


What is your budget (in USD)?  65
How much data do you need? (e.g., 10GB, Unlimited):  10
How many SMS do you need? (e.g., 500, Unlimited):  500
Minimum required download speed (Mbps):  100
Minimum required upload speed (Mbps):  10
Do you need cloud storage? (Yes/No):  Yes
How much hotspot data do you need? (e.g., 5GB, Unlimited):  Unlimited



Here are the top 3 recommended plans for you:
```json
[
    {
        "Brand": "StraightTalk",
        "Plan Name": "Platinum Unlimited",
        "Monthly Cost (USD)": 65,
        "Data Allowance": "Unlimited",
        "Talk Time": "Unlimited",
        "SMS": "Unlimited",
        "Max Download Speed (Mbps)": 195,
        "Max Upload Speed (Mbps)": 31,
        "Cloud Storage": "Yes",
        "Hotspot Data (GB)": "Unlimited",
        "Plan Description": "Platinum plan under StraightTalk brand with unlimited data, talktime, SMS access along with unlimited HOTSPOT data"
    },
    {
        "Brand": "TotalWireless",
        "Plan Name": "Total5G+ Unlimited",
        "Monthly Cost (USD)": 65,
        "Data Allowance": "Unlimited",
        "Talk Time": "Unlimited",
        "SMS": "Unlimited",
        "Max Download Speed (Mbps)": 195,
        "Max Upload Speed (Mbps)": 31,
        "Cloud Storage": "No",
        "Hotspot Data (GB)": "Unlimited",
        "Plan Description": "Total 5G+ unlimite